# Assignment #1


## Greedy Motif Search

In [1]:
from Bio import SeqIO
import numpy as np
import math

In [65]:
def totalcount(x, pseudocount):
    x=list(x)
    counts = np.array([
        x.count('A'),
        x.count('T'),
        x.count('G'),
        x.count('C')
        ])
    finalcount = (counts + pseudocount)/(len(x) + 4*pseudocount)
    return finalcount


def profile(DNA, k, pos, log_odd=True):
    N = len(DNA)# or pos0
    x = np.empty((N, k), dtype=str) #matrix with k-mers
    W = np.empty((4, k)) #position weight matrix
    for i in range(N):
        x[i, :] = list(DNA[i][pos[i]:pos[i]+k])
    for j in range(k):
        W[:,j] = totalcount(x[:,j], pseudocount=1)
    if log_odd==True: # log-odds matrix
        W2 = np.log(W*4)/np.log(4)
    else: 
        W2 = W
    return W2

def llr(profile, s):
    ref = {
        'A':0,
        'T':1,
        'G':2,
        'C':3
    }
    llratio = 0
    for i in range(len(s)):
        llratio += profile[ref[s[i]], i]
    return llratio

def motifpos(profile, DNA): #Finds the motif with highest llr for a given profile(pwm)
    N = len(DNA)
    k = profile.shape[1]
    pos = np.empty(N, dtype=int)
    for i in range(N):
        maxllr = -math.inf
        for j in range(len(DNA[i])-k+1):
            llrval = llr(profile, DNA[i][j:j+k])
            if llrval > maxllr:
                maxllr = llrval
                pos[i] = j
    return pos  

def GreedySearch(DNA, k):
    N = len(DNA)
    pos = np.array([np.random.randint(0, len(DNA[i]) - k) for i in range(N)])
    ind=0
    while True:
        ind = ind +1
        old_pos = pos.copy()
        pwm = profile(DNA, k, pos)
        pos = motifpos(pwm, DNA)
        print(pos)
        if np.array_equal(old_pos, pos):
            print(f'#iterations:{ind}')
            break
    return pos

In [76]:
strong = [str(record.seq) for record in SeqIO.parse("synth_50_strong.fa", "fasta")]
weak = [str(record.seq) for record in SeqIO.parse("synth_50_weak.fa", "fasta")]

k=10
strong_pos = GreedySearch(strong, k)
print(f"{k}-mers for 'synth_50_strong.fa':")
print(f'POS = {strong_pos}')
for i in range(len(strong)):
    print(strong[i][strong_pos[i]:strong_pos[i]+k])


[16 33 17 14  7 38 11  6 24 17]
[16 33 17 14  7 33 11  6 24 17]
[16 33 17 14  7 33 11  6 24 17]
#iterations:3
10-mers for 'synth_50_strong.fa':
POS = [16 33 17 14  7 33 11  6 24 17]
GTCGAAATGA
AGCGAAATGA
TACGAAATGA
TACGAAATGA
TTCGAAATGA
GTCGAAATGA
ACCGAAATGA
AGCGAAATGA
AGCGAAATGA
GGCGAAATGA


In [72]:

weak_pos = GreedySearch(weak, k)
print(f"{k}-mers for 'synth_50_weak.fa':")
print(f'POS = {weak_pos}') 
for i in range(len(weak)):
    print(weak[i][weak_pos[i]:weak_pos[i]+k])

[37  7 24 24 32 25 32 21  1  6]
[ 3  7 24 24 32 25 32  4  1  6]
[ 3  7 24 24 32 25 32  4  1  6]
#iterations:3
10-mers for 'synth_50_weak.fa':
POS = [ 3  7 24 24 32 25 32  4  1  6]
AAAATCGCTT
AGATTATGCT
AGTATCACCC
AGGACTTCCA
CAAACGTCGT
AGATTCTGCC
CGGATCCCAC
AGTATTGCCT
AGGATTGACC
AGGTCCTCCA


## Gibbs Sampler

In [ ]:
#functions

def RandomPOS(DNA):
    N = len(DNA); l = len(DNA[0]) #number of seqs, length of each seq
    randpos = np.random.uniform(0, l-1, N)
    return randpos

def totalcount(x, pseudocount):
    x=list(x)
    counts = np.array([
        x.count('A'),
        x.count('T'),
        x.count('G'),
        x.count('C')
        ])
    finalcount = (counts + pseudocount)/(len(x) + 4*pseudocount)
    return finalcount


def Profile(DNA, k, pos, log_odd=True):
    N = len(DNA)# or pos0
    x = np.empty((N, k), dtype=str) #matrix with k-mers
    W = np.empty((4, k)) #position weight matrix
    for i in range(N):
        x[i, :] = list(DNA[i][pos[i]:pos[i]+k])
    for j in range(k):
        W[:,j] = totalcount(x[:,j], pseudocount=1)
    if log_odd==True: # log-odds matrix
        W2 = np.log(W*4)/np.log(4)
    else: 
        W2 = W
    return W2

def Gibbs(DNA, k):
    randpos = RandomPOS(DNA)
    pwm = Profile(DNA, k, randpos)